### Загрузка бибилиотек

In [2]:
import pandas as pd


### Очистка данных

In [3]:
# Загрузка данных из CSV файлов
df1 = pd.read_csv('sales_inventory_orders_data.csv')
df2 = pd.read_csv('broken_sales.csv')

# Объединение датафреймов
data = pd.concat([df1, df2], ignore_index=True)

In [6]:
# Пример: Удаление дубликатов
data.drop_duplicates(inplace=True)

In [8]:
# Очистка и обработка пропущенных значений
data.dropna(subset=['Sales'], inplace=True)

In [10]:
# Пример: Преобразование формата даты
data['Date'] = pd.to_datetime(data['Date'])

In [13]:
# Создание сводных таблиц
# Total Price = Sales * Sale Price
data['TotalPrice'] = data['Sales'] * data['SalePrice']

In [14]:
data.shape

(3285, 11)

In [15]:
data.head()

,Date,Product,Sales,Orders,Inventory,UnrealizedDemand,DeliveryTime,DeliveryPrice,StoragePrice,SalePrice,TotalPrice
0,2023-01-01,Product_1,22.0,120,84,0,2,4.45,0.29,20,440.0
1,2023-01-02,Product_1,17.0,0,67,0,0,0.00,0.29,20,340.0
2,2023-01-03,Product_1,15.0,0,172,0,0,0.00,0.29,20,300.0
3,2023-01-04,Product_1,19.0,0,153,0,0,0.00,0.29,20,380.0
4,2023-01-05,Product_1,17.0,0,136,0,0,0.00,0.29,20,340.0


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3285 entries, 0 to 3284
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              3285 non-null   datetime64[ns]
 1   Product           3285 non-null   object        
 2   Sales             3285 non-null   float64       
 3   Orders            3285 non-null   int64         
 4   Inventory         3285 non-null   int64         
 5   UnrealizedDemand  3285 non-null   int64         
 6   DeliveryTime      3285 non-null   int64         
 7   DeliveryPrice     3285 non-null   float64       
 8   StoragePrice      3285 non-null   float64       
 9   SalePrice         3285 non-null   int64         
 10  TotalPrice        3285 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(5), object(1)
memory usage: 308.0+ KB


In [17]:
# Ежемесячные продажи для каждого продукта
monthly_sales = data.groupby(['Product', data['Date'].dt.to_period("M")])['Sales'].sum().reset_index()

# Ежемесячная выручка для каждого продукта
monthly_revenue = data.groupby(['Product', data['Date'].dt.to_period("M")])['TotalPrice'].sum().reset_index()


In [18]:
monthly_sales

,Product,Date,Sales
0,Product_1,2023-01,563.0
1,Product_1,2023-02,565.0
2,Product_1,2023-03,637.0
3,Product_1,2023-04,684.0
4,Product_1,2023-05,677.0
...,...,...,...
103,Product_9,2023-08,775.0
104,Product_9,2023-09,734.0
105,Product_9,2023-10,743.0
106,Product_9,2023-11,808.0


In [19]:
monthly_revenue

,Product,Date,TotalPrice
0,Product_1,2023-01,11260.0
1,Product_1,2023-02,11300.0
2,Product_1,2023-03,12740.0
3,Product_1,2023-04,13680.0
4,Product_1,2023-05,13540.0
...,...,...,...
103,Product_9,2023-08,62775.0
104,Product_9,2023-09,59454.0
105,Product_9,2023-10,60183.0
106,Product_9,2023-11,65448.0


### Модели оптимизации 

#### Модель экономического заказа (EOQ)

In [20]:
# Расчет EOQ для каждого продукта
def calculate_eoq(row):
    D = row['Sales'].sum()  # Годовой спрос (сумма продаж)
    S = row['DeliveryPrice']  # Затраты на заказ
    H = row['StoragePrice']  # Затраты на хранение за единицу продукции
    EOQ = np.sqrt((2 * D * S) / H)
    return EOQ

# Применение функции к каждому продукту
data['EOQ'] = data.groupby('Product').apply(calculate_eoq).reset_index(level=0, drop=True)

# Вывод результатов
print(data[['Product', 'EOQ']].drop_duplicates())


NameError: name 'np' is not defined